## basic use

the next cells show how the %cache magic should be used


Note: This notebook requires the the packages scikit-learn, numpy and cache-magic to be installed

In [22]:
import cache_magic

# delete everthing currently cached
%cache --reset 

# store a new value for a
%cache a = "111"

# fetch the cached value for a
%cache a = "111"

Resetting entire cache
Creating new value for variable 'a'
Loading cached value for variable 'a'. Time since caching: 0:00:00.000577


In [23]:
# an examle for an actual use-case
import cache_magic
import numpy as np
from sklearn import svm

%cache --reset

%timeit -n 1 -r 5 %cache -v 1 clf = svm.LinearSVC(dual='auto').fit(np.random.randint(5, size=(5000, 40)), np.random.randint(5, size=(5000)))


Resetting entire cache
Creating new value for variable 'clf'
Loading cached value for variable 'clf'. Time since caching: 0:00:00.000658
Loading cached value for variable 'clf'. Time since caching: 0:00:00.001776
Loading cached value for variable 'clf'. Time since caching: 0:00:00.002597
Loading cached value for variable 'clf'. Time since caching: 0:00:00.003328
The slowest run took 112.46 times longer than the fastest. This could mean that an intermediate result is being cached.
17.5 ms ± 33.4 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [24]:
# the following 4 cases use the same version

%cache -r

# without explicit version, the expression (=right hand site of assignment) is used as version
%cache a = 0

# if parameter is an integer, it will be the version
%cache -v 0 a = 1 

# if parameter is a variable name, it's value is used as version
my_version = 0
%cache -v my_version a = 1

# new and old version are converted into a string before comparing them
my_version_2 = "0"
%cache -v my_version_2 a = 1

Resetting entire cache
Creating new value for variable 'a'
Warning! Expression has changed since last save, which was at 2024-02-14 04:32:05.811509
To store a new value, change or omit the version ('-v' or '--version').
Loading cached value for variable 'a'. Time since caching: 0:00:00.001323
Warning! Expression has changed since last save, which was at 2024-02-14 04:32:05.811509
To store a new value, change or omit the version ('-v' or '--version').
Loading cached value for variable 'a'. Time since caching: 0:00:00.002876
Warning! Expression has changed since last save, which was at 2024-02-14 04:32:05.811509
To store a new value, change or omit the version ('-v' or '--version').
Loading cached value for variable 'a'. Time since caching: 0:00:00.004112


In [25]:
# show everything, that is cached
%cache

# generate some variables
%cache b=3
def fun(x):
    return x+1
%cache c = fun(b)
%cache -v c d = fun(1.1)

# show the new cache
%cache

Var Name,Size(MB),Stored Date,Version,Expression
a,1.3e-05,2024-02-14 04:32:05.811509,0,0
Total Size:,1.3e-05,,,


Creating new value for variable 'b'
Creating new value for variable 'c'
Creating new value for variable 'd'


Var Name,Size(MB),Stored Date,Version,Expression
a,1.3e-05,2024-02-14 04:32:05.811509,0,0
b,1.3e-05,2024-02-14 04:32:05.835470,0,3
c,1.3e-05,2024-02-14 04:32:05.838417,0,fun(b)
d,2.2e-05,2024-02-14 04:32:05.840963,4,fun(1.1)
Total Size:,6.1e-05,,,


## power use

the next cells show how the %cache magic can be used

In [26]:
import cache_magic
import numpy as np
from sklearn import svm
%cache --reset

# even if the expression changes, but not the version, the old value will still be loaded 
# in which case there will be a warning
%cache -v 1 clf = svm.LinearSVC(dual='auto').fit(np.random.randint(5, size=(1000, 40)), np.random.randint(5, size=(1000)))
%cache -v 1 clf = "not a classifier"
print(clf.predict(np.random.randint(5,size=(1,40)))[0])

# without an expression, it will always try to reload the cached value
del clf
%cache -v 1 clf
print(clf.predict(np.random.randint(5,size=(1,40)))[0])

# you can store the current value of a var without an actual statement by assigning it to itself
clf="not a classifier"
%cache -v 2 clf=clf
print(clf)


Resetting entire cache
Creating new value for variable 'clf'
Warning! Expression has changed since last save, which was at 2024-02-14 04:32:05.877668
To store a new value, change or omit the version ('-v' or '--version').
Loading cached value for variable 'clf'. Time since caching: 0:00:00.001843
0
Loading cached value for variable 'clf'. Time since caching: 0:00:00.003435
2
Creating new value for variable 'clf'
not a classifier


In [27]:
# while the cache still exists in the file system, the cell can be executed alone
import cache_magic

%cache clf
print(clf)


Loading cached value for variable 'clf'. Time since caching: 0:00:00.006313
not a classifier


In [28]:
# the cache is stored in the directory where the kernel was first started in 

import cache_magic
import os

%cache -r
%cache a=1
%cache b=1
%cache c=1

%cache
for root, dirs, files in os.walk(".cache-magic"):
    # there is one folder per cache variable
    print(root)

# if the working dir changes, the .cache-dir stays where it is
%cd ..
%cache
for root, dirs, files in os.walk(".cache-magic"):
    # no output, because no .cache-dir
    print(root)
    

%cd -
%cache
for root, dirs, files in os.walk(".cache-magic"):
    # now we see the cache directory againg
    print(root)
       

resetting entire cache
creating new value for variable 'a'
creating new value for variable 'b'
creating new value for variable 'c'


var name,size(byte),stored at date,version,expression(hash)
b,5,2018-01-25 15:06:28.438061,1,1
a,5,2018-01-25 15:06:28.437203,1,1
c,5,2018-01-25 15:06:28.438906,1,1


.cache_magic
.cache_magic/b
.cache_magic/a
.cache_magic/c
/home/jovyan


var name,size(byte),stored at date,version,expression(hash)
b,5,2018-01-25 15:06:28.438061,1,1
a,5,2018-01-25 15:06:28.437203,1,1
c,5,2018-01-25 15:06:28.438906,1,1


/home/jovyan/magic


var name,size(byte),stored at date,version,expression(hash)
b,5,2018-01-25 15:06:28.438061,1,1
a,5,2018-01-25 15:06:28.437203,1,1
c,5,2018-01-25 15:06:28.438906,1,1


.cache_magic
.cache_magic/b
.cache_magic/a
.cache_magic/c


In [29]:
import cache_magic

# always store a new value and never read from cache
%cache -r a=1

# remove a single variable from cache
%cache -r a

# Error:
%cache a

# load last value if possible, and store new value on miss
%cache a = a


Resetting cached values for a
Creating new value for variable 'a'
Resetting cached values for a
Error: Variable 'a' not in cache.
Creating new value for variable 'a'


In [30]:
# load last value if possible, but don't store new value on miss
import cache_magic

del a
%cache a

Loading cached value for variable 'a'. Time since caching: 0:00:00.014347


In [31]:
# You can use this magic-module as a regular python module

from cache_magic import CacheCall
cache = CacheCall(get_ipython().kernel.shell)

# setting all parameter by name
cache(
    version="*",
    reset=False,
    var_name="aaa",
    var_value="1+1",
    show_all=False,
    set_debug=True)

# setting all parameter by ordering
cache("1",False,"bbb","1+1",False, False)

# setting parameter selectivly
cache(show_all=True)


creating new value for variable 'aaa'
creating new value for variable 'bbb'


var name,size(byte),stored at date,version,expression(hash)
b,5,2018-01-25 15:06:28.438061,1,1
a,5,2018-01-25 15:06:28.565980,a,a
aaa,5,2018-01-25 15:06:28.704483,1+1,1+1
c,5,2018-01-25 15:06:28.438906,1,1
bbb,5,2018-01-25 15:06:28.706382,1,1+1


## development tests

the next cells show how the %cache magic should not be used

these examples are for debug-purposes only

In [32]:
#testing successfull calls

import cache_magic

# Dev-Note: use reload, so you don't have to restart the kernel everytime you change the 
from importlib import reload
reload(cache_magic)

my_version = 3
%cache --reset

print("   exptecting: new values")
%cache -v 2 a = "ex3"
%cache -v my_version c = "ex3"
%cache --version my_version sadsda = "ex3"
%cache -v 3 a=""
%cache -v 3 -r a=""
%cache -v 3 -r a=""


print("   exptecting: warnings")
%cache -v 3 a= " _ "
%cache -v 3 sadsda = "ex4"

print("   exptecting: stored values")
%cache -v my_version sadsda = "ex3"
%cache -v 3 sadsda = "ex3"


%cache magic is now registered in jupyter
Resetting entire cache
   exptecting: new values
Creating new value for variable 'a'
Creating new value for variable 'c'
Creating new value for variable 'sadsda'
Creating new value for variable 'a'
Resetting cached values for a
Creating new value for variable 'a'
Resetting cached values for a
Creating new value for variable 'a'
   exptecting: warnings
Warning! Expression has changed since last save, which was at 2024-02-14 04:32:06.082800
To store a new value, change or omit the version ('-v' or '--version').
Loading cached value for variable 'a'. Time since caching: 0:00:00.006609
Warning! Expression has changed since last save, which was at 2024-02-14 04:32:06.060966
To store a new value, change or omit the version ('-v' or '--version').
Loading cached value for variable 'sadsda'. Time since caching: 0:00:00.032757
   exptecting: stored values
Loading cached value for variable 'sadsda'. Time since caching: 0:00:00.034006
Loading cached value 

In [33]:
# testing errors

import cache_magic
reload(cache_magic)

# wrong versions
try:
    %cache -v "a" a = "ex3"
except cache_magic.CacheCallException as e:
    print("Exception was caught: " + str(e))

try:
    %cache -v "1" a
except cache_magic.CacheCallException as e:
    print("Exception was caught: " + str(e))

# wrong python statement
try:
    %cache -v a 1=a
except cache_magic.CacheCallException as e:
    print("Exception was caught: " + str(e))

%cache magic is now registered in jupyter
Error: Invalid version '"a"'. It must either be an Integer, *, or the name of a variable
Exception was caught: Invalid version '"a"'. It must either be an Integer, *, or the name of a variable
Error: Invalid version '"1"'. It must either be an Integer, *, or the name of a variable
Exception was caught: Invalid version '"1"'. It must either be an Integer, *, or the name of a variable
Error: Statement is not valid python: 1=a
 Error: cannot assign to literal here. Maybe you meant '==' instead of '='? (<unknown>, line 1)
Exception was caught: Statement is not valid python: 1=a
 Error: cannot assign to literal here. Maybe you meant '==' instead of '='? (<unknown>, line 1)


In [34]:
# testing loading without storing

import cache_magic
reload(cache_magic)

%cache --reset 
a=1
del a

# error: 
%cache a

%cache a=1
del a
%cache a
del a

# error:
try:
    %cache -v '1' a
except cache_magic.CacheCallException as e:
    print("Exception was caught: " + str(e))

# error
%cache -v 1 a

%cache --reset 
a=1
del a

# Error
%cache -v 0 a

%cache -v * a=1

# Error:
%cache -v * a

%cache magic is now registered in jupyter
Resetting entire cache
Error: Variable 'a' not in cache.
Creating new value for variable 'a'
Loading cached value for variable 'a'. Time since caching: 0:00:00.000599
Error: Invalid version ''1''. It must either be an Integer, *, or the name of a variable
Exception was caught: Invalid version ''1''. It must either be an Integer, *, or the name of a variable
Forced version '1' could not be found, instead found version '0'.If you don't care about a specific version, remove the version parameter.
Resetting entire cache
Error: Variable 'a' not in cache.
Creating new value for variable 'a'
Loading cached value for variable 'a'. Time since caching: 0:00:00.000931


In [35]:
# Nothing to do in this case
import cache_magic

%cache -v 1

In [36]:
import cache_magic

%cache -v 213 a = "1"
# get stored version via error message
%cache -v * a

# testing debug flag '-d'
%cache -d -v 1 -r a = "1"
%cache -d a = "1"

creating new value for variable 'a'
loading cached value for variable 'a'. Time since pickling  0:00:00.000334
resetting cached values for a
creating new value for variable 'a'
resetting because version mismatch
creating new value for variable 'a'


In [37]:
import cache_magic
from importlib import reload
reload(cache_magic)

%cache -r a=1
%cache -r a
%cache -r a=1

%cache magic is now registered in jupyter
Resetting cached values for a
Creating new value for variable 'a'
Resetting cached values for a
Resetting cached values for a
Creating new value for variable 'a'


In [38]:
import cache_magic
from importlib import reload
reload(cache_magic)

%cache -r a = 1
%cache a
print (a)

%cache magic is now registered in jupyter
Resetting cached values for a
Creating new value for variable 'a'
Loading cached value for variable 'a'. Time since caching: 0:00:00.002680
1


In [39]:
import cache_magic
from importlib import reload
reload(cache_magic)

def foo(x):
    return x+1

%cache --reset
%cache -v * a= foo(3)
%cache -v * a= foo(3)
%cache -v * a= 2


%cache magic is now registered in jupyter
Resetting entire cache
Creating new value for variable 'a'
Loading cached value for variable 'a'. Time since caching: 0:00:00.000840
Expression has changed since last save, which was at 2024-02-14 04:32:06.383014
Creating new value for variable 'a'


In [40]:
#!pip install -e .